In [7]:
import torch
import matplotlib.pyplot as plt
import torch.nn as nn
from numpy.random import standard_t
from sklearn.preprocessing import StandardScaler
from torchinfo import summary


In [16]:
linear=nn.Linear(5,3)
nn.init.uniform_(linear.weight)
print(linear.weight.data)

tensor([[0.6303, 0.4463, 0.2627, 0.0409, 0.2022],
        [0.9133, 0.7225, 0.1711, 0.0032, 0.8158],
        [0.3323, 0.8213, 0.1706, 0.8245, 0.3978]])


In [17]:
linear=nn.Linear(5,3)
nn.init.constant_(linear.weight,5)
print(linear.weight.data)

tensor([[5., 5., 5., 5., 5.],
        [5., 5., 5., 5., 5.],
        [5., 5., 5., 5., 5.]])


In [18]:
linear=nn.Linear(5,3)
nn.init.zeros_(linear.weight)
print(linear.weight.data)

tensor([[0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.]])


In [19]:
linear=nn.Linear(5,3)
nn.init.ones_(linear.weight)
print(linear.weight.data)

tensor([[1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1.]])


In [20]:
linear=nn.Linear(5,3)
nn.init.normal_(linear.weight,mean=0,std=1)
print(linear.weight.data)

tensor([[-0.1196,  0.5444,  1.2949, -0.2346, -0.8818],
        [ 0.4551,  1.2848,  0.1806,  0.4005, -0.8524],
        [ 0.0649, -0.1144, -1.1852, -1.3571, -1.3947]])


In [25]:
class Model(nn.Module):
    def __init__(self):
        super(Model,self).__init__()
        self.hidden1=nn.Linear(3,3)
        nn.init.xavier_normal_(self.hidden1.weight)
        self.hidden2=nn.Linear(3,2)
        nn.init.kaiming_normal_(self.hidden2.weight)
        self.output=nn.Linear(2,2)

    # ✅ forward 必须与 init 同级缩进！
    def forward(self,x):
        x=self.hidden1(x)
        x=torch.sigmoid(x)
        x=self.hidden2(x)
        x=torch.relu(x)
        x=self.output(x)
        x=torch.softmax(x,dim=-1)
        return x

In [26]:
# 实例化model对象
if __name__=='__main__':
    device = torch.device("mps" if torch.backends.mps.is_available() else "cpu")
    model=Model().to(device)
# 随机产生数据
    my_data=torch.randn(5,3).to(device)
    print(my_data.shape)
# 数据经过神经网络模型训练
    output=model(my_data)
    print(output.shape)
# 计算模型参数
    summary(model,input_size=(3,),device=device)
# 计算每层每个神经元的w和b个数总和
    print("======查看模型参数w和b======")
    for name, parameter in model.named_parameters():
     print(name, parameter)
# 查看模型参数

torch.Size([5, 3])
torch.Size([5, 2])
======查看模型参数w和b======
hidden1.weight Parameter containing:
tensor([[ 0.1105, -0.1715, -0.0543],
        [-0.0180,  1.6179, -0.4338],
        [-0.3672, -0.2303, -0.3315]], device='mps:0', requires_grad=True)
hidden1.bias Parameter containing:
tensor([-0.3063,  0.0671, -0.3668], device='mps:0', requires_grad=True)
hidden2.weight Parameter containing:
tensor([[-0.5045,  0.5238, -1.1293],
        [-0.1864, -1.1667,  0.1769]], device='mps:0', requires_grad=True)
hidden2.bias Parameter containing:
tensor([-0.5233, -0.0211], device='mps:0', requires_grad=True)
output.weight Parameter containing:
tensor([[-0.5119,  0.1797],
        [-0.2698, -0.3115]], device='mps:0', requires_grad=True)
output.bias Parameter containing:
tensor([0.1812, 0.1655], device='mps:0', requires_grad=True)


In [ ]:
#价格分类案例

In [5]:
from torch.utils.data import TensorDataset,DataLoader
import torch.nn as nn
import torch.optim as optim
from sklearn.datasets import make_regression
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import time

In [3]:
def create_dataset():
    #  创建数据集
    data=pd.read_csv('手机价格预测.csv')
    #特征值和标签
    x,y=data.iloc[:,:-1],data.iloc[:,-1]
    #类型转换，特征值，目标值
    x=x.astype(np.float32)

    y=y.astype(np.int64)
    # 数据集划分
    X_train,X_test,y_train,y_test=train_test_split(x,y,test_size=0.2,random_state=42)
    # 构建数据集,转换为pytorch的形式
    train_dataset=TensorDataset(torch.from_numpy(X_train.values),torch.tensor(y_train.values))
    test_dataset=TensorDataset(torch.from_numpy(X_test.values),torch.tensor(y_test.values))
    # 返回结果
    return train_dataset,test_dataset,X_train.shape[1], len(np.unique(y))

In [ ]:
#获取数据的结果

In [8]:
if __name__ == '__main__':
    train_dataset,test_dataset,input_dim,class_num=create_dataset()
    print(input_dim)
    print(class_num)

20
4


In [10]:
class PhonePriceModel(nn.Module):
    def __init__(self, input_dim, class_num):
        super(PhonePriceModel, self).__init__()
        self.Linear1 = nn.Linear(input_dim, 128)
        self.bn1 = nn.BatchNorm1d(128)
        self.Linear2 = nn.Linear(128, 256)
        self.bn2 = nn.BatchNorm1d(256)
        self.Linear3 = nn.Linear(256, class_num)
        nn.init.kaiming_normal_(self.Linear1.weight, nonlinearity='relu')
        nn.init.kaiming_normal_(self.Linear2.weight, nonlinearity='relu')
        nn.init.xavier_normal_(self.Linear3.weight)
    def forward(self, x):
        x = torch.selu_(self.bn1(self.Linear1(x)))
        x = torch.selu_(self.bn2(self.Linear2(x)))
        output = self.Linear3(x)
        return output


In [11]:
#模型实例化
if __name__ == '__main__':
    model=PhonePriceModel(input_dim,class_num)
    summary(model, input_size=(16, input_dim))

In [12]:
def train(train_dataset, input_dim, class_num):
    device = torch.device("mps" if torch.backends.mps.is_available() else "cpu")
    model = PhonePriceModel(input_dim, class_num).to(device)
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.AdamW(model.parameters(), lr=0.001, weight_decay=1e-4)

    # 使用 ReduceLROnPlateau 调度器（监控验证集 loss）
    scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=5)
    correct = 0
    epochs = 80
    for epoch in range(epochs):
        model.train()
        train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
        total_loss = 0

        for x, y in train_loader:
            x,y= x.to(device), y.to(device)
            output = model(x)
            loss = criterion(output, y)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            total_loss += loss.item()

        avg_train_loss = total_loss / len(train_loader)

        # 验证集评估
        model.eval()
        val_loader = DataLoader(test_dataset, batch_size=16, shuffle=False)
        val_loss = 0
        correct = 0
        total = 0

        with torch.no_grad():
            for x_val, y_val in val_loader:
                x_val, y_val = x_val.to(device), y_val.to(device)
                val_output = model(x_val)
                loss = criterion(val_output, y_val)
                val_loss += loss.item()

                # ✅ 验证精度统计
                y_pred = torch.argmax(val_output, dim=1)
                correct += (y_pred == y_val).sum().item()
                total += y_val.size(0)

        avg_val_loss = val_loss / len(val_loader)


        # 使用验证集 loss 调整学习率
        scheduler.step(avg_val_loss)
        current_lr = optimizer.param_groups[0]['lr']
        print(
            f"第{epoch + 1}轮 | 训练损失: {avg_train_loss:.4f} | 验证损失: {avg_val_loss:.4f}  | 当前学习率: {current_lr:.6f}")
    val_accuracy = correct / total
    print(f"验证精度: {val_accuracy:.4f}")
    # 保存模型
    torch.save(model.state_dict(), '手机价格预测.pth')

In [13]:
#模型训练
if __name__ == '__main__':
    train_dataset,test_dataset,input_dim,class_num=create_dataset()
    train(train_dataset, input_dim, class_num)

第1轮 | 训练损失: 0.8231 | 验证损失: 0.5123  | 当前学习率: 0.001000
第2轮 | 训练损失: 0.5897 | 验证损失: 0.3453  | 当前学习率: 0.001000
第3轮 | 训练损失: 0.4770 | 验证损失: 0.2719  | 当前学习率: 0.001000
第4轮 | 训练损失: 0.4849 | 验证损失: 0.2536  | 当前学习率: 0.001000
第5轮 | 训练损失: 0.4596 | 验证损失: 0.2452  | 当前学习率: 0.001000
第6轮 | 训练损失: 0.5326 | 验证损失: 0.3227  | 当前学习率: 0.001000
第7轮 | 训练损失: 0.4700 | 验证损失: 0.2492  | 当前学习率: 0.001000
第8轮 | 训练损失: 0.4934 | 验证损失: 0.2557  | 当前学习率: 0.001000
第9轮 | 训练损失: 0.4742 | 验证损失: 0.2310  | 当前学习率: 0.001000
第10轮 | 训练损失: 0.4563 | 验证损失: 0.2922  | 当前学习率: 0.001000
第11轮 | 训练损失: 0.4755 | 验证损失: 0.3378  | 当前学习率: 0.001000
第12轮 | 训练损失: 0.4566 | 验证损失: 0.2430  | 当前学习率: 0.001000
第13轮 | 训练损失: 0.4197 | 验证损失: 0.2727  | 当前学习率: 0.001000
第14轮 | 训练损失: 0.4669 | 验证损失: 0.2058  | 当前学习率: 0.001000
第15轮 | 训练损失: 0.5236 | 验证损失: 0.2889  | 当前学习率: 0.001000
第16轮 | 训练损失: 0.3911 | 验证损失: 0.2392  | 当前学习率: 0.001000
第17轮 | 训练损失: 0.4616 | 验证损失: 0.2218  | 当前学习率: 0.001000
第18轮 | 训练损失: 0.5270 | 验证损失: 0.2558  | 当前学习率: 0.001000
第19轮 | 训练损失: 0.5335 | 验证损失: 0.2402  |

In [231]:
#使用训练好的模型，对未知的样本的进行预测的过程。我们这里使用前面单独划分出来的验证集来进行评估
def test(test_dataset, input_dim, class_num):
    import os
    # 加载模型和训练好的网络参数
    model=PhonePriceModel(input_dim, class_num)
    model.load_state_dict(torch.load('手机价格预测.pth'))
    # 构建加载器
    dataLoader=DataLoader(test_dataset, batch_size=16, shuffle=False)
    # 评估测试集
    correct=0
    # 遍历测试集中的数据
    for x,y in dataLoader:
    # 将其送入网络中
        output=model(x)
    # 获取类别结果
        y_pred=torch.argmax(output,dim=1)
    # 获取预测正确的个数
        correct+=(y_pred==y).sum().item()
    # 求预测精度
    print(f'预测精度为{correct/len(test_dataset):.4f}')
    # 获取数据
if __name__ == '__main__':
    _, test_dataset, input_dim, class_num = create_dataset()
    test(test_dataset, input_dim, class_num)

预测精度为0.8025
